In [1]:
pip install google-search-results

## Google Shopping Scraper (Paginated + Cleaned)

This script scrapes Google Shopping results using **SerpApi** with pagination support.  

**Features:**
- Fetches up to 5 pages of products (configurable with `MAX_PAGES`).
- Extracts **title, price (INR), source, and direct merchant link**.
- Deduplicates results (avoids repeating the same product).
- Adds small delays between pages to respect API usage.
- Saves results into a **CSV file** (`google_shopping_results.csv`) with proper encoding so `₹` is displayed correctly in Excel.

**CSV Columns:**
- Title  
- Price (INR)  
- Source (merchant name)  
- Link (direct product link)


In [ ]:
from serpapi import GoogleSearch
import csv
import time

API_KEY = Your_api_key      
SEARCH_TERM = "Nothing 45W charger"        
CSV_FILE = "google_shopping_results.csv"   
MAX_PAGES = 5                              
SLEEP_BETWEEN_PAGES = 1.2                   

# Country/language/currency for India
GL = "in"
HL = "en"
CURRENCY = "INR"


def normalize_price(p: str) -> str:
    """Return price string as-is but ensure it's a string (handle None)."""
    return (p or "").strip()

def get_direct_link(product: dict) -> str:
    """
    Prefer the merchant's direct product link over Google redirect links.
    SerpApi often provides:
      - product_link: merchant page (preferred)
      - link: sometimes Google redirect (fallback)
    """
    link = product.get("product_link") or product.get("link") or ""
    return link.strip()

def extract_items(products):
    """Yield cleaned items (title, price, source, link) from SerpApi items."""
    for p in products:
        title = (p.get("title") or "").strip()
        price = normalize_price(p.get("price"))
        source = (p.get("source") or "").strip()
        link = get_direct_link(p)

        # Skip obviously bad/empty links
        if not link:
            continue

        yield {
            "title": title,
            "price": price,
            "source": source,
            "link": link,
        }

def scrape_google_shopping(query: str):
    """
    Paginate through Google Shopping results via SerpApi.
    Returns a list of unique items.
    """
    results_all = []
    seen_links = set()

    start = 0
    page = 1

    while page <= MAX_PAGES:
        params = {
            "engine": "google_shopping",
            "api_key": API_KEY,
            "q": query,
            "gl": GL,                 # country
            "hl": HL,                 # language
            "currency": CURRENCY,     # show prices in INR
            "start": start,           # pagination offset
        }

        print(f"🔎 Fetching page {page} (start={start}) …")
        search = GoogleSearch(params)
        data = search.get_dict()

        # SerpApi returns shopping results in this key
        products = data.get("shopping_results", []) or []

        # Sometimes items appear under 'inline_shopping_results' (rare)
        if not products and "inline_shopping_results" in data:
            products = data["inline_shopping_results"]

        if not products:
            print("✅ No more products found. Stopping.")
            break

        count_before = len(results_all)
        for item in extract_items(products):
            if item["link"] not in seen_links:
                seen_links.add(item["link"])
                results_all.append(item)

        added = len(results_all) - count_before
        print(f"🛒 Found {len(products)} raw items; added {added} unique items.")

        # Next page
        page += 1
        start += len(products) 
        time.sleep(SLEEP_BETWEEN_PAGES)

    return results_all

def save_csv(items, filename):
    # Use utf-8-sig to display ₹ correctly in Excel
    with open(filename, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow(["Title", "Price (INR)", "Source", "Link"])
        for it in items:
            writer.writerow([it["title"], it["price"], it["source"], it["link"]])
    print(f"✅ Saved {len(items)} items to '{filename}'")


if __name__ == "__main__":
    items = scrape_google_shopping(SEARCH_TERM)
    if not items:
        print("❌ No products extracted. Try a broader query or check your API key/quota.")
    else:
        save_csv(items, CSV_FILE)


🔎 Fetching page 1 (start=0) …
🛒 Found 40 raw items; added 40 unique items.
🔎 Fetching page 2 (start=40) …
🛒 Found 22 raw items; added 22 unique items.
🔎 Fetching page 3 (start=62) …
✅ No more products found. Stopping.
✅ Saved 62 items to 'google_shopping_results.csv'


## Google Shopping Scraper (Single Page, Quick Version)

This is a **simpler scraper** that fetches only the **first page** of Google Shopping results for a given query.  

**Features:**
- Extracts **title, price, source, and link** from the first page only.
- Uses fewer API calls (good for saving quota).
- Saves data into `shopping_results.csv`.

**CSV Columns:**
- Title  
- Price  
- Link  
- Source


In [ ]:
from serpapi import GoogleSearch
import csv

API_KEY = Your_api_key
SEARCH_TERM = "Nothing 45W charger"

params = {
    "engine": "google_shopping",
    "api_key": API_KEY,
    "q": SEARCH_TERM,
    "gl": "in",     # country India
    "hl": "en",     # language English
    "currency": "INR"
}

search = GoogleSearch(params)
results = search.get_dict()
products = results.get("shopping_results", [])

with open("shopping_results.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["Title", "Price", "Link", "Source"])
    for p in products:
        title = p.get("title", "")
        price = p.get("price", "")
        link = p.get("product_link") or p.get("link", "")  # direct link preferred
        source = p.get("source", "")
        writer.writerow([title, price, link, source])

print(f"✅ Saved {len(products)} items to shopping_results.csv")


✅ Saved 40 items to shopping_results.csv
